<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Carga_de_datos_CSV_XLSX_VersionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MONITOREO INSTRUMENTACIÓN PRESAS

#**Carga Masiva de datos Crudos**

In [9]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PUNTOS_FIJOS_MI (CSV)


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,06/10/2021,Izquierda,PF01,-0.001,-0.002,0.001333,0.002236,2.236068,243.434949,-0.100000,-0.200000,0.133333,0.223607
1,10/10/2021,Izquierda,PF01,-0.008,0.003,0.000333,0.008544,8.544004,159.443955,-0.571429,0.214286,0.023810,0.610286
2,19/10/2021,Izquierda,PF01,-0.012,0.004,0.001333,0.012649,12.649111,161.565051,-0.521739,0.173913,0.057971,0.549961
3,28/10/2021,Izquierda,PF01,0.001,0.001,-0.000667,0.001414,1.414214,45.000000,0.031250,0.031250,-0.020833,0.044194
4,02/11/2021,Izquierda,PF01,-0.007,0.004,-0.002667,0.008062,8.062258,150.255119,-0.189189,0.108108,-0.072072,0.217899


#📥 Descarga individual de DataFrames por instrumento
Esta sección permite seleccionar un tipo de instrumento y el formato de origen de los datos (CSV o XLSX), y descargar individualmente el DataFrame correspondiente cargado previamente.

Funcionamiento:

Seleccioná el instrumento desde el primer desplegable (por ejemplo, piezometros_electricos, inclinometros, etc.).

Elegí el formato de origen de los datos: csv o xlsx.

Presioná el botón "📥 Descargar seleccionado" para obtener el archivo.

In [10]:
import tempfile
import os
from IPython.display import FileLink
try:
    from google.colab import files
except ImportError:
    files = None

# Función para exportar y descargar un único DataFrame
def descargar_individual(instrumento, origen):
    with output:
        clear_output(wait=True)
        print(f"⏳ Generando archivo para: {instrumento.upper()} ({origen.upper()})")

        if origen == "csv":
            df = datos_csv[instrumento]
            ext = "csv"
        else:
            df = datos_xlsx[instrumento]
            ext = "xlsx"

        if df.empty:
            print("⚠️ No hay datos para este instrumento y origen.")
            return

        # Crear archivo temporal
        temp_dir = tempfile.mkdtemp()
        file_path = os.path.join(temp_dir, f"{instrumento}.{ext}")

        # Guardar archivo
        if ext == "csv":
            df.to_csv(file_path, index=False)
        else:
            df.to_excel(file_path, index=False)

        print("✅ Archivo listo para descargar:")
        if files:  # Google Colab
            files.download(file_path)
        else:  # Jupyter local
            display(FileLink(file_path, result_html_prefix=f"📎 Descargar {ext.upper()}: "))

# Dropdowns + botón
dropdown_instrumento = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='40%')
)

dropdown_origen = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='30%')
)

boton_individual = widgets.Button(
    description='📥 Descargar seleccionado',
    button_style='primary',
    layout=widgets.Layout(width='30%')
)

def on_click_descarga(b):
    descargar_individual(dropdown_instrumento.value, dropdown_origen.value)

boton_individual.on_click(on_click_descarga)

# Mostrar descarga individual
descarga_individual_ui = widgets.HBox([dropdown_instrumento, dropdown_origen, boton_individual])
display(descarga_individual_ui)
